### Using Scispacy

In [1]:
### chose to do this from cli
#%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

In [2]:
import pandas as pd

In [30]:
patient_notes = pd.read_csv("patient_notes.csv")
features = pd.read_csv("features.csv")

print(patient_notes[patient_notes["case_num"] == 0][:10])
features[features["case_num"] == 0][:10]

   pn_num  case_num                                         pn_history
0       0         0  17-year-old male, has come to the student heal...
1       1         0  17 yo male with recurrent palpitations for the...
2       2         0  Dillon Cleveland is a 17 y.o. male patient wit...
3       3         0  a 17 yo m c/o palpitation started 3 mos ago; \...
4       4         0  17yo male with no pmh here for evaluation of p...
5       5         0  17 yo m, presenting with palpitations/increase...
6       6         0  17 yo male presents with 3-4 month history of ...
7       7         0  17 yo male presents with a 2-3 month history o...
8       8         0  HPI: 17 YO complains of intermittent heart pal...
9       9         0  Mr. Cleveland is a 17 yo M who presents to cli...


,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure
3,3,0,Intermittent-symptoms
4,4,0,Lightheaded
5,5,0,No-hair-changes-OR-no-nail-changes-OR-no-tempe...
6,6,0,Adderall-use
7,7,0,Shortness-of-breath
8,8,0,Caffeine-use
9,9,0,heart-pounding-OR-heart-racing


In [4]:
case_num_zero = patient_notes["pn_history"][patient_notes["case_num"] == 0][:1500]
whole_case_num_zero = "\n".join(case_num_zero.to_list())
len(whole_case_num_zero)
whole_case_num_zero[:1000]

"17-year-old male, has come to the student health clinic complaining of heart pounding. Mr. Cleveland's mother has given verbal consent for a history, physical examination, and treatment\r\n-began 2-3 months ago,sudden,intermittent for 2 days(lasting 3-4 min),worsening,non-allev/aggrav\r\n-associated with dispnea on exersion and rest,stressed out about school\r\n-reports fe feels like his heart is jumping out of his chest\r\n-ros:denies chest pain,dyaphoresis,wt loss,chills,fever,nausea,vomiting,pedal edeam\r\n-pmh:non,meds :aderol (from a friend),nkda\r\n-fh:father had MI recently,mother has thyroid dz\r\n-sh:non-smoker,mariguana 5-6 months ago,3 beers on the weekend, basketball at school\r\n-sh:no std\n17 yo male with recurrent palpitations for the past 3 mo lasting about 3 - 4 min, it happened about 5 - 6 times since the beginning. One time durign a baskeball game two days ago light headedness, pressure in the chest, catching breath, but no fainting. During teh episodes no sweating.

In [19]:
import prepare_jag
clean_note = prepare_jag.basic_clean3(whole_case_num_zero)
clean_note = prepare_jag.remove_stopwords(clean_note, exclude_words=["no"])
len(clean_note)

968544

### run this always

In [16]:
import scispacy
import spacy

from scispacy.linking import EntityLinker

sci_nlp = spacy.load("en_ner_bc5cdr_md")
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
print("works, yay!")

works, yay!


### From Scispacy GitHub

In [ ]:
# # NOTE: The resolve_abbreviations parameter is optional, and requires that
# # the AbbreviationDetector pipe has already been added to the pipeline. Adding
# # the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# # that linking will only be performed on the long form of abbreviations.
# sci_nlp.add_pipe("scispacy_linker", config={"linker_name": "umls"})

# doc = sci_nlp(clean_note)
# # Let's look at a random entity!
# # doc comes from above
# entity = doc.ents[1]
# print(f"Name: {entity}")

# # Each entity is linked to UMLS with a score
# # (currently just char-3gram matching).
# linker = nlp.get_pipe("scispacy_linker")
# for umls_ent in entity._.kb_ents:
# 	print(linker.kb.cui_to_entity[umls_ent[0]])

### From https://gbnegrini.com/post/biomedical-text-nlp-scispacy-named-entity-recognition-medical-records/

In [ ]:
doc = sci_nlp(clean_note)
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

- case num 0 patient notes

In [26]:
case_num_zero_notes = patient_notes[patient_notes["case_num"] == 0]
case_num_zero_notes.head(3)

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...


- case num 0 feature text

In [31]:
case_num_zero_features = features[features["case_num"] == 0]
case_num_zero_features.head(3)

,feature_num,case_num,feature_text
0,0,0,Family-history-of-MI-OR-Family-history-of-myoc...
1,1,0,Family-history-of-thyroid-disorder
2,2,0,Chest-pressure


### merge of the notes and features

In [32]:
notes_features = patient_notes.merge(right=features, left_on="case_num", right_on="case_num")#.drop("case_num_y", axis=1).rename({"case_num_x": "case_num"}, axis=1)
print(notes_features.shape)
notes_features.head()

(626902, 5)


,pn_num,case_num,pn_history,feature_num,feature_text
0,0,0,"17-year-old male, has come to the student heal...",0,Family-history-of-MI-OR-Family-history-of-myoc...
1,0,0,"17-year-old male, has come to the student heal...",1,Family-history-of-thyroid-disorder
2,0,0,"17-year-old male, has come to the student heal...",2,Chest-pressure
3,0,0,"17-year-old male, has come to the student heal...",3,Intermittent-symptoms
4,0,0,"17-year-old male, has come to the student heal...",4,Lightheaded


### Create function to repeat this for each note in the dataframe

In [33]:
def get_doc_ents(clean_text):
    
    doc = sci_nlp(clean_text)
    # print("TEXT", "START", "END", "ENTITY TYPE")
    return {ent.text: [ent.start_char, ent.end_char] for ent in doc.ents}

In [34]:
notes_features["clean_pn_history"] = notes_features["pn_history"].apply(prepare_jag.basic_clean3).apply(prepare_jag.remove_stopwords, exclude_words=["no"])

notes_features["clean_notes_history_entities"] = notes_features["clean_pn_history"].apply(get_doc_ents)

In [ ]:
notes_features.head()